# Librerias

In [51]:
import os
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Shapefile de estaciones climatologicas

In [52]:
path_Estaciones = r"D:\UAT_Doc\Climatologia_Tomates_Tamos\Estaciones.shp"

# Rasters de PCP

In [53]:
path1960to1969 = r"D:\UAT_Doc\Sobre_proyecto_de_investigacion\worldclim\1960-1969"

In [54]:
list1960to1969 = [os.path.join (path1960to1969, band) for band in os.listdir(path1960to1969) if band.endswith('tif')]
print(list1960to1969)

['D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-01.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-02.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-03.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-04.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-05.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-06.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-07.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-08.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-09.tif', 'D:\\UAT_Doc\\Sobre_proyecto_de_investigacion\\worldclim\\1960-1969\\wc2.1_2.5m_prec_1960-10.tif', 'D:\\UAT_

## Verificar las rutas

In [55]:
for raster_path in list1960to1969:
    if not os.path.exists(raster_path):
        print(f'el archivo {raster_path} no existe.')
        raise FileNotFoundError (f'El archivo {raster_path} no existe.')

In [56]:
Estaciones = gpd.read_file(path_Estaciones)
Estaciones

,Estacion,Nom_Est,Mun,Estado,Situacion,Lat,Long,geometry
0,30166,Tamos,Panuco,Veracruz,Operando,22.216111,-97.998611,POINT (-97.99861 22.21611)
1,28055,Los Tomates,Altamira,Tamaulipas,Operando,22.452778,-98.197222,POINT (-98.19722 22.45278)


In [59]:
listEst = [nom for nom in Estaciones.Nom_Est]
listEst

['Tamos', 'Los Tomates']

In [72]:
dfs_por_Estacion = {} 
for Est in listEst:
    dfs_por_Estacion[Est] = pd.DataFrame(columns=['año_mes','PCP_mm'])

In [58]:
coordenadas = [(geom.x, geom.y) for geom in Estaciones.geometry]
coordenadas

[(-97.99861111, 22.21611111), (-98.19722222, 22.45277778)]

In [74]:
path1960to1969xls = r"D:\UAT_Doc\Sobre_proyecto_de_investigacion\worldclim\PCP2_1960_1969.xlsx"

# Extraccion de valores

In [75]:
for raster_path in list1960to1969:
    with rasterio.open(raster_path) as raster:
        raster_name2 = os.path.splitext(os.path.basename(raster_path))[0]
        raster_name = raster_name2.split('_')[-1]
    
        for Est, coordenada in zip(listEst, coordenadas):
            PCP = raster.sample([coordenada]) #Obteniendo el generador
            pcp = list(PCP) #Convirtiendo el generador en lista
            
            if pcp and pcp[0]:
                dfs_por_Estacion[Est] = dfs_por_Estacion[Est].append({'año_mes':raster_name,'PCP_mm':pcp[0][0]}, ignore_index=True)
          
    
    
with pd.ExcelWriter(path1960to1969xls) as writer:
    for Est, df_Est in dfs_por_Estacion.items():
        df_Est.to_excel(writer, sheet_name=Est, index=False)
        

In [71]:
dfs_por_Estacion

{'Los Tomates':      año_mes     PCP_mm
 0    1960-01  11.400000
 1    1960-01   8.100000
 2    1960-02   7.400000
 3    1960-02   5.800000
 4    1960-03   7.700000
 ..       ...        ...
 475  1969-10  87.800003
 476  1969-11  24.400000
 477  1969-11  24.000000
 478  1969-12  22.299999
 479  1969-12  13.800000
 
 [480 rows x 2 columns]}

# Exportar a Excel

In [ ]:
df.to_excel(path1960to1969xls, index = False)
print(f'los valores han sido extraidos y enviados a {path1960to1969xls}')